In [59]:
from opt.mc_sim import *
from common.variables import *
from sim.sim_functions import *
import torch
import gym
import numpy as np
from gym import spaces
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete, Tuple, MultiBinary


In [60]:
s = SourcingEnv()

In [61]:
class CustomGymEnv(Env):

    def __init__(self, sourcing_env):
            self.SourcingEnv = sourcing_env
            self.counter = 0
    
            # Actions we can take
            self.action_space = MultiDiscrete([INVEN_LIMIT,INVEN_LIMIT])
            
            # Inventory Observation State
            self.observation_space = Box(low=np.array([-30, 0, 0, 0, 0]), high=np.array([30, 30, 30, 1, 1]), shape=(5,), dtype=int)
                                                   
        
    def step(self, action):
                                            
            next_state, event, i, event_probs, supplier_index = self.SourcingEnv.step(action)
            self.counter += 1
            
            if (next_state.s) >=0: 
                reward = -float((2 * next_state.s) + np.sum(np.multiply(action, PROCUREMENT_COST_VEC)))
            else: 
                reward = -float((-10* next_state.s) + np.sum(np.multiply(action, PROCUREMENT_COST_VEC)))
            
            info = {}
            
            if self.counter < PERIODS:
                done = False
                
            else:
                done = True
                print([action])
                      
            
            next_state_array = np.array(next_state.get_list_repr())
            return next_state_array, reward, done, info
        
            
        
    def reset(self):
            self.SourcingEnv = SourcingEnv()
            return np.array(self.SourcingEnv.current_state.get_list_repr())
        
           

In [62]:
s = SourcingEnv()
custom_gym_env = CustomGymEnv(s)

In [63]:
episodes = 10
for episode in range(1, episodes+1):
    state = custom_gym_env.reset()
    done = False
    cost = 0
    #info = 0


    
    
    while not done:
        
        action = custom_gym_env.action_space.sample()
        n_state, reward, done, info = custom_gym_env.step(action)
        cost=reward
        observation = custom_gym_env.step(action)
        #info = [action]
        #print([action])
       #lista = np.load("Test.npz") 
        #lista = lista['arr_0']
    
      
    print('Episode:{} Cost:{} Observation:{} info:{}' .format(episode, cost, observation, info))   #Observation: {}(, observation)
custom_gym_env.close()

[array([25, 26])]
[array([10, 25])]
[array([10, 25])]
Episode:1 Cost:-1185.0 Observation:(array([400,  20,  25,   1,   1]), -1375.0, True, {}) info:{}
[array([5, 2])]
[array([5, 2])]
Episode:2 Cost:-235.0 Observation:(array([ 2, 10,  2,  1,  1]), -239.0, True, {}) info:{}
[array([ 0, 11])]
[array([ 0, 11])]
Episode:3 Cost:-55.0 Observation:(array([11,  0, 11,  1,  1]), -77.0, True, {}) info:{}
[array([10, 27])]
[array([10, 27])]
Episode:4 Cost:-585.0 Observation:(array([10, 10, 54,  1,  1]), -605.0, True, {}) info:{}
[array([ 6, 16])]
[array([ 6, 16])]
Episode:5 Cost:-350.0 Observation:(array([ 6,  6, 32,  1,  1]), -362.0, True, {}) info:{}
[array([23, 10])]
[array([23, 10])]
Episode:6 Cost:-1085.0 Observation:(array([23, 23, 10,  1,  0]), -1131.0, True, {}) info:{}
[array([18, 22])]
[array([18, 22])]
Episode:7 Cost:-920.0 Observation:(array([22, 36, 22,  1,  1]), -964.0, True, {}) info:{}
[array([6, 2])]
[array([6, 2])]
Episode:8 Cost:-280.0 Observation:(array([-1, 12,  4,  1,  1]), -

In [72]:
str(custom_gym_env.SourcingEnv.current_state.s)

'8.0'

In [75]:
if (custom_gym_env.SourcingEnv.current_state.s) >=0: 
    reward = -((2 * custom_gym_env.SourcingEnv.current_state.s) + np.sum(np.multiply(action, PROCUREMENT_COST_VEC)))
else: 
    reward = -((-10* custom_gym_env.SourcingEnv.current_state.s) + np.sum(np.multiply(action, PROCUREMENT_COST_VEC)))

In [76]:
reward

-61.0

In [64]:
from stable_baselines3 import A2C, DDPG, DQN, PPO, TD3, SAC
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [65]:
log_path = os.path.join('Backup', 'Tests')

In [66]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.1, device='cuda', gamma=0.8)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [67]:
model.learn(total_timesteps=100)

Logging to Backup/Tests/PPO_4
[array([14, 26])]
[array([20, 20])]
[array([24,  2])]
[array([19, 27])]
[array([20, 22])]
[array([17, 12])]
[array([ 3, 12])]
[array([20, 19])]
[array([23, 23])]
[array([26, 20])]
[array([24,  4])]
[array([ 6, 10])]
[array([26, 28])]
[array([18, 18])]
[array([ 0, 19])]
[array([23, 22])]
[array([0, 6])]
[array([10, 21])]
[array([15, 16])]
[array([21, 19])]
[array([ 0, 29])]
[array([29, 24])]
[array([1, 6])]
[array([17, 14])]
[array([8, 9])]
[array([ 0, 19])]
[array([17,  1])]
[array([ 0, 11])]
[array([16, 11])]
[array([ 9, 10])]
[array([ 1, 13])]
[array([14, 21])]
[array([11, 25])]
[array([23,  0])]
[array([22,  4])]
[array([11, 23])]
[array([2, 3])]
[array([ 1, 16])]
[array([6, 0])]
[array([17, 16])]
[array([19,  5])]
[array([11, 15])]
[array([ 5, 25])]
[array([ 2, 27])]
[array([16,  9])]
[array([19, 16])]
[array([17, 21])]
[array([19, 18])]
[array([27, 27])]
[array([2, 9])]
[array([22, 16])]
[array([26, 15])]
[array([ 1, 23])]
[array([20, 10])]
[array([11

[array([7, 6])]
[array([26, 27])]
[array([19,  2])]
[array([20, 10])]
[array([25,  2])]
[array([10, 29])]
[array([13,  2])]
[array([11, 10])]
[array([29, 17])]
[array([16,  6])]
[array([26, 22])]
[array([ 3, 22])]
[array([25,  7])]
[array([7, 9])]
[array([13,  2])]
[array([14, 16])]
[array([17, 16])]
[array([20, 17])]
[array([19,  2])]
[array([26, 17])]
[array([ 3, 20])]
[array([24, 13])]
[array([3, 6])]
[array([28,  2])]
[array([18,  1])]
[array([16,  2])]
[array([14, 22])]
[array([17,  6])]
[array([4, 4])]
[array([23,  9])]
[array([16, 24])]
[array([20, 27])]
[array([0, 3])]
[array([13,  0])]
[array([13, 25])]
[array([ 5, 24])]
[array([20, 22])]
[array([19, 27])]
[array([14,  7])]
[array([16, 14])]
[array([19,  8])]
[array([10, 18])]
[array([7, 9])]
[array([18,  7])]
[array([27, 26])]
[array([27,  8])]
[array([21, 11])]
[array([25, 27])]
[array([14,  8])]
[array([17, 28])]
[array([28, 10])]
[array([28, 24])]
[array([7, 1])]
[array([13, 16])]
[array([ 5, 22])]
[array([21, 28])]
[array

[array([3, 6])]
[array([17, 28])]
[array([ 1, 25])]
[array([ 3, 20])]
[array([7, 8])]
[array([11, 13])]
[array([24,  2])]
[array([6, 0])]
[array([13, 12])]
[array([21, 27])]
[array([13, 27])]
[array([7, 0])]
[array([28, 22])]
[array([4, 9])]
[array([21, 29])]
[array([16, 28])]
[array([20, 10])]
[array([23, 17])]
[array([14, 27])]
[array([17, 29])]
[array([15, 25])]
[array([ 9, 24])]
[array([ 5, 18])]
[array([24, 22])]
[array([1, 4])]
[array([26, 10])]
[array([4, 4])]
[array([4, 7])]
[array([14, 18])]
[array([16, 25])]
[array([10, 27])]
[array([9, 5])]
[array([18, 21])]
[array([25, 23])]
[array([15,  4])]
[array([27, 26])]
[array([0, 6])]
[array([17, 21])]
[array([1, 5])]
[array([12, 14])]
[array([ 4, 18])]
[array([28,  4])]
[array([6, 1])]
[array([19, 21])]
[array([25,  0])]
[array([29,  1])]
[array([27, 28])]
[array([13,  7])]
[array([14, 25])]
[array([29, 28])]
[array([24,  6])]
[array([2, 0])]
[array([15, 20])]
[array([22, 16])]
[array([12,  8])]
[array([ 5, 10])]
[array([11,  2])]


[array([7, 6])]
[array([21, 18])]
[array([ 0, 20])]
[array([27, 29])]
[array([19, 11])]
[array([28, 26])]
[array([22, 23])]
[array([0, 6])]
[array([20, 16])]
[array([15, 27])]
[array([26, 11])]
[array([11, 11])]
[array([18,  5])]
[array([10,  8])]
[array([25, 21])]
[array([7, 6])]
[array([27, 11])]
[array([27, 26])]
[array([ 9, 11])]
[array([ 4, 29])]
[array([15, 24])]
[array([20, 22])]
[array([12, 18])]
[array([22, 13])]
[array([29, 13])]
[array([13, 13])]
[array([23,  4])]
[array([ 1, 15])]
[array([28, 24])]
[array([21,  7])]
[array([17, 16])]
[array([4, 2])]
[array([22,  6])]
[array([13, 14])]
[array([22,  8])]
[array([13, 25])]
[array([28, 12])]
[array([2, 1])]
[array([ 7, 29])]
[array([24,  8])]
[array([4, 0])]
[array([14,  4])]
[array([25, 14])]
[array([15, 24])]
[array([18, 20])]
[array([11,  6])]
[array([ 7, 25])]
[array([8, 3])]
[array([10,  2])]
[array([ 1, 29])]
[array([ 5, 11])]
[array([0, 3])]
[array([ 9, 14])]
[array([10, 26])]
[array([19,  8])]
[array([ 1, 23])]
[array([

In [36]:
model.predict(np.array([-6, 0, 0, 1, 1]))

(array([2, 1]), None)

In [37]:
model.predict(np.array([10, 0, 0, 1, 1]))

(array([2, 1]), None)

In [38]:
model.predict(np.array([100, 0, 0, 1, 1]))

(array([2, 1]), None)

In [39]:
model.predict(np.array([100, 0, 0, 1, 1]))

(array([2, 1]), None)

In [12]:
m_state = custom_gym_env.SourcingEnv.current_state

str(m_state)

'Stock: 0, n backorders: [0. 0.], supplier status (on/off): [1. 1.]'

In [ ]:
custom_gym_env.observation_space

# manual tests

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(next_state_array)

In [ ]:
reward

In [ ]:
custom_gym_env.reset()

In [ ]:
from stable_baselines3.common.env_checker import check_env

In [ ]:
check_env(custom_gym_env) #warn=True)

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = custom_gym_env.reset()
    done = False
    cost = 0
    info = 0

    
    
    while not done:
        
        action = custom_gym_env.action_space.sample()
        n_state, reward, done, info = custom_gym_env.step(action)
        cost=reward
        observation = custom_gym_env.step(action)
        info = [action] #shows sample of action, but not which was taken...
    print('Episode:{} Cost:{} Observation:{} Info:{}' .format(episode, cost, observation, info))   #Observation: {}(, observation)
custom_gym_env.close()

In [ ]:
next_state_array, reward, done, info = custom_gym_env.step(np.array([0, 0]))

In [ ]:
str(info)

In [ ]:
custom_gym_env.

In [ ]:
custom_gym_env.action_space([])

In [7]:
from stable_baselines3 import A2C, DDPG, DQN, PPO, TD3, SAC
import os



In [8]:
env = custom_gym_env

In [ ]:
log_path = os.path.join('Backup', 'A2C')

In [ ]:
from stable_baselines3 import A2C
policy_kwargs = dict(net_arch=[dict(pi=[256, 256, 128], vf=[256, 256, 128])])
model = A2C("MlpPolicy", env, verbose=1,  learning_rate=(0.001),\
 n_steps=50, gamma=0.8, gae_lambda=1.0, ent_coef=0.2, vf_coef=0.2, \
tensorboard_log=log_path, seed=50, device='cuda' )
model.learn(total_timesteps=1000)

model.learn(total_timesteps=10)

import gym
from stable_baselines3 import A2C
policy_kwargs = dict(activation_fn=th.nn.ReLU, net_arch=[dict(pi=[256, 256, 128], vf=[256, 256, 128])])
model = A2C("MlpPolicy", env, verbose=1,  learning_rate=linear_schedule(0.001),\ n_steps=50, gamma=0.8, gae_lambda=1.0, ent_coef=0.2, vf_coef=0.2, \ tensorboard_log=logdir,  policy_kwargs=policy_kwargs, seed=50, device='cuda' )
model.learn(total_timesteps=50000)

In [ ]:
model.save('A2C')

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, custom_gym_env, n_eval_episodes=10, render=False)

custom_gym_env.observation_space

In [9]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [12]:
log_path = os.path.join('Dict')

In [13]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.01, device='cuda', gamma=0.9)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
model.learn(total_timesteps=100000)

Logging to Dict/PPO_1
Stock: -1, n backorders: [24.  4.], supplier status (on/off): [1. 1.] [array([24,  4])]
Stock: 23.0, n backorders: [15. 32.], supplier status (on/off): [1. 1.] [array([15, 28])]
Stock: 22.0, n backorders: [22. 57.], supplier status (on/off): [1. 1.] [array([ 7, 25])]
Stock: 44.0, n backorders: [29. 75.], supplier status (on/off): [1. 1.] [array([29, 18])]
Stock: 73.0, n backorders: [19. 90.], supplier status (on/off): [1. 1.] [array([19, 15])]
Stock: 92.0, n backorders: [  8. 109.], supplier status (on/off): [1. 1.] [array([ 8, 19])]
Stock: 100.0, n backorders: [ 11. 130.], supplier status (on/off): [1. 1.] [array([11, 21])]
Stock: 230.0, n backorders: [39. 13.], supplier status (on/off): [1. 1.] [array([28, 13])]
Stock: 269.0, n backorders: [26. 31.], supplier status (on/off): [1. 1.] [array([26, 18])]
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -699     |
| time/              |     

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -35       |
| time/                   |           |
|    fps                  | 843       |
|    iterations           | 11        |
|    time_elapsed         | 26        |
|    total_timesteps      | 22528     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.19e-06 |
|    explained_variance   | nan       |
|    learning_rate        | 0.01      |
|    loss                 | 0         |
|    n_updates            | 100       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0         |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -35       |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -35       |
| time/                   |           |
|    fps                  | 838       |
|    iterations           | 21        |
|    time_elapsed         | 51        |
|    total_timesteps      | 43008     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.19e-06 |
|    explained_variance   | nan       |
|    learning_rate        | 0.01      |
|    loss                 | 0         |
|    n_updates            | 200       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0         |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -35       |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -35       |
| time/                   |           |
|    fps                  | 836       |
|    iterations           | 31        |
|    time_elapsed         | 75        |
|    total_timesteps      | 63488     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.19e-06 |
|    explained_variance   | nan       |
|    learning_rate        | 0.01      |
|    loss                 | 0         |
|    n_updates            | 300       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0         |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -35       |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -35       |
| time/                   |           |
|    fps                  | 830       |
|    iterations           | 41        |
|    time_elapsed         | 101       |
|    total_timesteps      | 83968     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.19e-06 |
|    explained_variance   | nan       |
|    learning_rate        | 0.01      |
|    loss                 | 0         |
|    n_updates            | 400       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0         |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -35       |


In [17]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.1, device='cuda', gamma=0.9)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [18]:
model.learn(total_timesteps=100000)

Logging to Dict/PPO_2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -752     |
| time/              |          |
|    fps             | 1205     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -60       |
| time/                   |           |
|    fps                  | 981       |
|    iterations           | 2         |
|    time_elapsed         | 4         |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 63.446995 |
|    clip_fraction        | 0.952     |
|    clip_range           | 0.2       |
|    entropy_loss         | -3.52     |
|    explained_variance   | 0         |
|    learning_rate        | 0.1       

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -60       |
| time/                   |           |
|    fps                  | 848       |
|    iterations           | 12        |
|    time_elapsed         | 28        |
|    total_timesteps      | 24576     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000392 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.179     |
|    n_updates            | 110       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.476     |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -60       |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -60       |
| time/                   |           |
|    fps                  | 837       |
|    iterations           | 22        |
|    time_elapsed         | 53        |
|    total_timesteps      | 45056     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000392 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.342     |
|    n_updates            | 210       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.476     |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -60       |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -55       |
| time/                   |           |
|    fps                  | 833       |
|    iterations           | 32        |
|    time_elapsed         | 78        |
|    total_timesteps      | 65536     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -8.18e-08 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.00794   |
|    n_updates            | 310       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.872     |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -55       |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -55       |
| time/                   |           |
|    fps                  | 829       |
|    iterations           | 42        |
|    time_elapsed         | 103       |
|    total_timesteps      | 86016     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -8.18e-08 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.192     |
|    n_updates            | 410       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.871     |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -55       |


In [19]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, device='cuda', gamma=0.9)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [20]:
model.learn(total_timesteps=100000)

Logging to Dict/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -748     |
| time/              |          |
|    fps             | 1222     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -650        |
| time/                   |             |
|    fps                  | 993         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015019368 |
|    clip_fraction        | 0.118       |
|    clip_range           | 0.2         |
|    entropy_loss         | -6.8        |
|    explained_variance   | -2.38e-07   |
|    lea

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1           |
|    ep_rew_mean          | -145        |
| time/                   |             |
|    fps                  | 859         |
|    iterations           | 11          |
|    time_elapsed         | 26          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.063992515 |
|    clip_fraction        | 0.61        |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.25       |
|    explained_variance   | -1.19e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 1.15e+04    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.112      |
|    value_loss           | 4.18e+04    |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1       

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean          | -2.7       |
| time/                   |            |
|    fps                  | 835        |
|    iterations           | 21         |
|    time_elapsed         | 51         |
|    total_timesteps      | 43008      |
| train/                  |            |
|    approx_kl            | 0.24523228 |
|    clip_fraction        | 0.921      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.07      |
|    explained_variance   | 0          |
|    learning_rate        | 0.0003     |
|    loss                 | 23.8       |
|    n_updates            | 200        |
|    policy_gradient_loss | -0.127     |
|    value_loss           | 485        |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1          |
|    ep_rew_mean

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 834      |
|    iterations           | 31       |
|    time_elapsed         | 76       |
|    total_timesteps      | 63488    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00377 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 2.44e-15 |
|    n_updates            | 300      |
|    policy_gradient_loss | 0        |
|    value_loss           | 5.8e-15  |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                   |          |
|    fps                  | 833      |
|    iterations           | 41       |
|    time_elapsed         | 100      |
|    total_timesteps      | 83968    |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -0.00149 |
|    explained_variance   | nan      |
|    learning_rate        | 0.0003   |
|    loss                 | 0.00202  |
|    n_updates            | 400      |
|    policy_gradient_loss | 0        |
|    value_loss           | 0.00276  |
--------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1        |
|    ep_rew_mean          | 0        |
| time/                  

In [22]:
output_obj

AttributeError: 'PPO' object has no attribute 'output_obj'

In [ ]:
model.save('PPO')

In [ ]:
evaluate_policy(model, custom_gym_env, n_eval_episodes=10, render=False)

In [ ]:
dir(model)

In [23]:
model.__dict__

{'policy_class': stable_baselines3.common.policies.ActorCriticPolicy,
 'device': device(type='cpu'),
 'env': <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv at 0x7fd15b622dc0>,
 '_vec_normalize_env': None,
 'verbose': 1,
 'policy_kwargs': {},
 'observation_space': Box([-30   0   0   0   0], [30 30 30  1  1], (5,), int64),
 'action_space': MultiDiscrete([30 30]),
 'n_envs': 1,
 'num_timesteps': 100352,
 '_total_timesteps': 100000,
 '_num_timesteps_at_start': 0,
 'eval_env': None,
 'seed': None,
 'action_noise': None,
 'start_time': 1670408708005698198,
 'policy': ActorCriticPolicy(
   (features_extractor): FlattenExtractor(
     (flatten): Flatten(start_dim=1, end_dim=-1)
   )
   (mlp_extractor): MlpExtractor(
     (shared_net): Sequential()
     (policy_net): Sequential(
       (0): Linear(in_features=5, out_features=64, bias=True)
       (1): Tanh()
       (2): Linear(in_features=64, out_features=64, bias=True)
       (3): Tanh()
     )
     (value_net): Sequential(
      

In [ ]:
model._last_obs

In [ ]:
model.vf_coef

In [ ]:
model.observation_space

In [ ]:
model.predict(np.array([-6, 0, 0, 1, 1]))

In [ ]:
custom_gym_env.reset()

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 10, batch_size=50)
model.learn(total_timesteps=10000)

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 10)
model.learn(total_timesteps=10000)

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 100)
model.learn(total_timesteps=10000)

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 100, batch_size=50)
model.learn(total_timesteps=10000)

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50)
model.learn(total_timesteps=10000)

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50)
model.learn(total_timesteps=100000)

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50, seed=10)
model.learn(total_timesteps=100000)

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50, seed=100)
model.learn(total_timesteps=100000)

In [ ]:
log_path = os.path.join('Backup','PPO')
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, n_steps= 1000, batch_size=50, seed=1000)
model.learn(total_timesteps=100000)

In [ ]:

torch.cuda.is_available()

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


In [ ]:
 torch.cuda.device_count()